In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np 
import numpy.random as npr
import scipy as sp
import scipy.linalg as sl
from scipy.fft import fft
from scipy.fft import ifft


La fonction barriere_reg renvoie un potentiel barrière avec une régularisation de sorte à ce que le potentiel soit dans $C^\infty_C$. 
L'idée est de rajouter à un potentiel barrière classique entre $s$ et $s+l$ des splins $C^\infty_C$. allant à $0$ en $\varepsilon$.
Cela régularise la solution et évite des problèmes éventuelles aussi bien théorique que numérique lorsque l'on utilise un potentiel discontinu.

In [ ]:
splin_exp = lambda x: np.exp(-1/x)*(x>0) / (np.exp(-1/x)*(x>0)+ np.exp(-1/(1-x))*(x>0))
raccord = lambda x,p1,p2,a,b: (1-splin_exp((x-a)/(b-a)))*p1 + splin_exp((x-a)/(b-a))*p2 

def barriere_reg(X,t,x1,x2,epsi):
    V=np.zeros(X.size)

    for i in range(X.size) :
        x = X[i]

        if(x<x1-epsi):                      V[i] = 0 
        elif((x1-epsi<x)  and (x<x1+epsi)): V[i] = raccord(x,0,V0,x1-epsi,x1+epsi)
        elif((x1+epsi<x)  and (x<x2-epsi)): V[i] = V0
        elif((x2-epsi<x)  and (x<x2+epsi)): V[i] = raccord(x,V0,0,x2-epsi,x2+epsi)
        elif(x2+epsi<x):                    V[i] = 0

    return V

La fonction $\textit{Kin(Nx)}$ renvoie le vecteur $[0,1,4,9..., (Nx/2)^2,- (Nx/2)^2, ...-1]$. Il pourra être utilisé lors du calcul de la dérivée seconde par TF. Elle pourra être remplacée par fftfreq par moment 

In [ ]:
def Kin (Nx):
    
    Nx_2 = int((Nx/2)*(Nx%2==0) + ((Nx-1)/2)*(Nx%2==1)) +1
    K = np.zeros((Nx), dtype='complex')
    for i in range(0,Nx_2-1):
        K[i] = 0.5*(2*np.pi/L)**2 *i*i
        print(i)
    for i in range(Nx_2,Nx-1):
        K[i] = 0.5*(2*np.pi/L)**2 *(i-Nx)*(i-Nx)
        print(i)
    return K

la fonction $\textit{Concentration de masse}$ va nous donner la norme de la fonction psi compris entre $a$ et $b$ en fonction des paramètres numérique $L$ et $Nx$ du problème.

In [ ]:
def Concentration_de_masse(psi,Nx,L,a,b):
    l = np.abs(b-a);    occupation  = l/(2*L) 
    an = int((np.abs(a+L)/(2*L))*Nx) ; bn = int((np.abs(b+L)/(2*L))*Nx)
    N = np.abs(bn-an)
    print(l,N,an,bn)
    psi_l = psi[an:bn]
    masse = np.sqrt(l/N)*np.linalg.norm(psi_l)
    return masse

la fonction $\textit{dynamics-fft-diss}$ calcul la solution de l'équation de schrödinger par méthode semi-spectral en espace et splitting en temps. 
la fonction sauvegarde en chaque pas de temps la solution $\psi_t$ et sa transformée de fourier $\phi_t = \mathcal{F}(\psi_t)$

In [ ]:
def dynamics_fft_diss(psi0_fun=(lambda x: np.exp(-x**2)), V_fun=(lambda x,t: 0), L=10, Nx=100, T=4, Nt=100):

    dt = T/Nt; dx = L/Nx
    # Kinetic = Kin(Nx)
    Kinetic = (0.5*(2*np.pi/L)**2) *np.fft.fftfreq(Nx, dx)*np.fft.fftfreq(Nx, dx)
    I = np.linspace(-L, L,Nx,endpoint=False)
    Psi_T = np.zeros((Nx,Nt), dtype="complex")
    Phi_T = np.zeros((Nx,Nt), dtype="complex")

    Psi_T[:,0]=psi0_fun(I)
    
    
    for i in range(1,Nt):
        ti = dt*i
        Phi_T[:,i] = (np.exp(-1j*Kinetic*dt)) * fft(np.exp(-1j*V_fun(I,ti)*dt) * Psi_T[:,i-1])
        Psi_T[:,i] = ifft(Phi_T[:,i])
        # print(ti,np.sqrt(L/Nx)*np.linalg.norm(Psi_T[:,i]))
    return Psi_T, Phi_T

La fonction $\textit{plot-psi}$ va crée une animation de notre simulation obtenu à partir de $\textit{dynamics-fft-diss}$

In [ ]:

def plot_psi(psi, duration=10, frames_per_second=30, L=10, show_potential=False):
    
    fig, ax = plt.subplots()
    t_data = np.linspace(0, 1, np.size(psi, 1)) # 1 is arbitrary here
    x_data = np.linspace(-L,L,  np.size(psi,0))
    # set the min and maximum values of the plot, to scale the axis
    m = min(0, np.min(np.real(psi)), np.min(np.imag(psi)))
    M = np.max(np.abs(psi))
    
    # set the axis once and for all
    ax.set(xlim=[-L,L], ylim=[m,M], xlabel='x', ylabel='psi')
    
    # dummy plots, to update during the animation
    real_plot = ax.plot(x_data, np.real(psi[:, 0]), label='Real')[0]
    imag_plot = ax.plot(x_data, np.imag(psi[:, 0]), label='Imag')[0]
    abs_plot  = ax.plot(x_data, np.abs(psi[:, 0]), label='Abs')[0]
    if(show_potential):V_plot  =   ax.plot(x_data, V(x_data,0), label='V')[0]
    ax.legend()

    # define update function as an internal function (that can access the variables defined before)
    # will be called with frame=0...(duration*frames_per_second)-1
    def update(frame):
        print(frame)
        # get the data by linear interpolation
        t = frame / (duration * frames_per_second)
        psi_t = np.array([np.interp(t, t_data, psi[i, :]) for i in range(np.size(psi,0))])
        # update the plots
        real_plot.set_ydata(np.real(psi_t))
        imag_plot.set_ydata(np.imag(psi_t))
        abs_plot.set_ydata(np.abs(psi_t))

    ani = animation.FuncAnimation(fig=fig, func=update, frames=duration*frames_per_second, interval=1000/frames_per_second)
    return ani


Développement de la barrière de potentiel: $\\ $
Avec la fonction $\textit{Barriere-reg}$, nous allons pouvoir comparer le bruit causé par un choc entre une onde incidente et un potentiel discontinu avec un choc entre une incidente et un potentiel $C^\infty$.
Avec la fonction $\textit{Concentration-de-masse}$, nous allons pouvoir étudier la masse qui est transmise par effet tunnel au travers de la barrière. $\\$ 

On note que cette étude peut s'étendre au cas du puit de potentiel lorsque l'on pose deux barrière des deux côtes de l'onde.

In [ ]:
a=1; kx=1000; x0=-3
psi0= lambda x: 1/(np.sqrt(2*np.pi *a*a)) *np.exp(-((x-x0)*(x-x0))/(2*a*a)) *np.exp(1j*kx*(x-x0))

L=20; Nx=2000; Nt=5000; T=50
l=0.2; s=4; V0=10; epsi=0.1

V = lambda x,t : V0*(x<=s+l )*(x>=s) + V0*(x>=-s)*(x<=-s-l)                     #barriere discontinue
V = lambda x,t : barriere_reg(x,t,s,s+l,epsi) + barriere_reg(x,t,-s-l,-s,epsi)  #barriere C infty
